In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os, shutil

Using TensorFlow backend.


In [2]:
base_dir = 'datasets/dogs-vs-cats'

original_dir = os.path.join(base_dir, 'original')
original_train_dir = os.path.join(original_dir, 'train')

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

cats_train_dir = os.path.join(train_dir, 'cats')
cats_validation_dir = os.path.join(validation_dir, 'cats')

dogs_train_dir = os.path.join(train_dir, 'dogs')
dogs_validation_dir = os.path.join(validation_dir, 'dogs')

In [3]:
os.mkdir(train_dir)
os.mkdir(validation_dir)
os.mkdir(cats_train_dir)
os.mkdir(cats_validation_dir)
os.mkdir(dogs_train_dir)
os.mkdir(dogs_validation_dir)

In [4]:
def copy_images_to_folder(filename_pattern, start_range, stop_range, src_dir, dst_dir):
    filenames = [filename_pattern.format(i) for i in range(start_range, stop_range)]
    for filename in filenames:
        src = os.path.join(src_dir, filename)
        dst = os.path.join(dst_dir, filename)
        shutil.copyfile(src, dst)

copy_images_to_folder('cat.{}.jpg', 0, 10000, original_train_dir, cats_train_dir)
copy_images_to_folder('cat.{}.jpg', 10000, 12500, original_train_dir, cats_validation_dir)

copy_images_to_folder('dog.{}.jpg', 0, 10000, original_train_dir, dogs_train_dir)
copy_images_to_folder('dog.{}.jpg', 10000, 12500, original_train_dir, dogs_validation_dir)

In [5]:
batch_size = 20

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(50,50), 
                                                    batch_size=batch_size, 
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(validation_dir, 
                                                              target_size=(50,50), 
                                                              batch_size=batch_size, 
                                                              class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 128)         147584    
__________

In [7]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit_generator(train_generator, 
                              steps_per_epoch=20000/batch_size, 
                              epochs=10, 
                              validation_data=validation_generator, 
                              validation_steps=5000/batch_size)

Epoch 1/10
1000/1000 [==============================] - 230s 230ms/step - loss: 0.6356 - acc: 0.6301 - val_loss: 0.5886 - val_acc: 0.6770
Epoch 2/10
1000/1000 [==============================] - 135s 135ms/step - loss: 0.5272 - acc: 0.7405 - val_loss: 0.4813 - val_acc: 0.7812
Epoch 3/10
1000/1000 [==============================] - 124s 124ms/step - loss: 0.4682 - acc: 0.7792 - val_loss: 0.4411 - val_acc: 0.7968
Epoch 4/10
1000/1000 [==============================] - 119s 119ms/step - loss: 0.4251 - acc: 0.8059 - val_loss: 0.4301 - val_acc: 0.8020
Epoch 5/10
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3831 - acc: 0.8322 - val_loss: 0.4598 - val_acc: 0.7926
Epoch 6/10
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3539 - acc: 0.8493 - val_loss: 0.4248 - val_acc: 0.8156
Epoch 7/10
1000/1000 [==============================] - 118s 118ms/step - loss: 0.3233 - acc: 0.8621 - val_loss: 0.3960 - val_acc: 0.8380
Epoch 8/10
1000/1000 [============